In [54]:
import pandas as pd
import numpy as np
from datetime import date


### Extract Columns List

In [55]:
with open('column_names.txt', 'r') as file:
    text = file.read()

Columns_List = text.split('\n')

Cols_List = [item.strip() for item in Columns_List if item.strip()]

len(Cols_List)


4073

### Initialize Single-Row Df

In [56]:
df=pd.DataFrame(columns=Cols_List)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Columns: 4073 entries, ScheduledArrTime to DepDelay
dtypes: object(4073)
memory usage: 132.0+ bytes


In [57]:
Date = date(2019,3,14)   
ScheduledArrTime  = 230
ScheduledDepTime   =568     
ScheduledElapsedTime  =28974
Airline_Code = "WN"
TailNum ="924WN"          
Org_Airport_Code ="LAX"      
Dest_Airport_Code = "ATL"          
Distance  =10000         


In [58]:
def OHE(df,Date,ScheduledArrTime,ScheduledDepTime,ScheduledElapsedTime,Airline_Code,TailNum,Org_Airport_Code,Dest_Airport_Code,Distance):
    df['ScheduledArrTime']=pd.Series(ScheduledArrTime)
    df['ScheduledDepTime']=pd.Series(ScheduledDepTime)
    df['ScheduledElapsedTime']=pd.Series(ScheduledElapsedTime)

    DP_Airline = "UniqueCarrier_"+Airline_Code
    df[DP_Airline]=pd.Series(1)

    df[TailNum]=pd.Series(1)

    DP_Origin = "Origin_"+Org_Airport_Code
    df[DP_Origin]=pd.Series(1)

    DP_Origin = "Dest_"+Dest_Airport_Code
    df[DP_Origin]=pd.Series(1)

    df["Distance"]=pd.Series(Distance)

    DP_Weekday = "DayOfWeek_"+str(Date.weekday())
    df[DP_Weekday]=pd.Series(1)

    df["Day"]=pd.Series(Date.day)

    DP_Month = "Month_"+str(Date.month)
    df[DP_Month]=pd.Series(1)
    df=df.fillna(0,inplace=True)
OHE(df,Date,ScheduledArrTime,ScheduledDepTime,ScheduledElapsedTime,Airline_Code,TailNum,Org_Airport_Code,Dest_Airport_Code,Distance)

In [59]:

df

,ScheduledArrTime,Distance,ScheduledDepTime,ScheduledElapsedTime,Day,DepTemperature,DepWindSpeed,DepWindDirection,DepPrecipitation,DepRain,...,DayOfWeek_5,DayOfWeek_6,DayOfWeek_7,Month_1,Month_2,Month_3,Month_4,Month_5,Month_6,DepDelay
0,230,10000,568,28974,14,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [60]:
Stats = pd.read_csv("Stats.csv")

In [61]:
def Normalize(Column,Value, Type ):
    if Type=="Z":
        row = Stats[Stats['Column_Name']==Column]
        mean = row['Mean'].iloc[0]
        sd = row['Standard_Deviation'].iloc[0]
        result = (Value-mean)/sd
        
    elif Type == "Log":
        result=np.log(np.abs(Value + 0.1) )
           
    else:
        row = Stats[Stats['Column_Name']==Column]
        min = row['Min'].iloc[0]
        max = row['Max'].iloc[0]
        result = (Value-min)/(max-min)
    return result

In [62]:
Nomralize_Dict = {'ScheduledArrTime':"Z",'ScheduledDepTime':"Z",'ScheduledElapsedTime':"Z",'Distance':"Log",'DepTemperature':"Z",'DepWindSpeed':"Log",'DepWindDirection':"Min",'DepPrecipitation':"Z",
                  'DepRain':"Z",'DepSnowFall':"Log",'ArrTemperature':"Z",'ArrWindSpeed':"Log",'ArrWindDirection':"Min",'ArrPrecipitation':"Z",'ArrRain':"Z",'ArrSnowFall':"Log",'Day':'Min'}

In [63]:
for key, value in Nomralize_Dict.items():
    print(key,df[key],value)
    result=Normalize(key,df[key],value)
    df[key]=result
df

ScheduledArrTime 0    230
Name: ScheduledArrTime, dtype: int64 Z
ScheduledDepTime 0    568
Name: ScheduledDepTime, dtype: int64 Z
ScheduledElapsedTime 0    28974
Name: ScheduledElapsedTime, dtype: int64 Z
Distance 0    10000
Name: Distance, dtype: int64 Log
DepTemperature 0    0
Name: DepTemperature, dtype: int64 Z
DepWindSpeed 0    0
Name: DepWindSpeed, dtype: int64 Log
DepWindDirection 0    0
Name: DepWindDirection, dtype: int64 Min
DepPrecipitation 0    0
Name: DepPrecipitation, dtype: int64 Z
DepRain 0    0
Name: DepRain, dtype: int64 Z
DepSnowFall 0    0
Name: DepSnowFall, dtype: int64 Log
ArrTemperature 0    0
Name: ArrTemperature, dtype: int64 Z
ArrWindSpeed 0    0
Name: ArrWindSpeed, dtype: int64 Log
ArrWindDirection 0    0
Name: ArrWindDirection, dtype: int64 Min
ArrPrecipitation 0    0
Name: ArrPrecipitation, dtype: int64 Z
ArrRain 0    0
Name: ArrRain, dtype: int64 Z
ArrSnowFall 0    0
Name: ArrSnowFall, dtype: int64 Log
Day 0    14
Name: Day, dtype: int64 Min


,ScheduledArrTime,Distance,ScheduledDepTime,ScheduledElapsedTime,Day,DepTemperature,DepWindSpeed,DepWindDirection,DepPrecipitation,DepRain,...,DayOfWeek_5,DayOfWeek_6,DayOfWeek_7,Month_1,Month_2,Month_3,Month_4,Month_5,Month_6,DepDelay
0,-3.051809,9.21035,-2.208409,256.269924,0.433333,-1.106722,-2.302585,-0.001771,-0.216732,-0.202053,...,0,0,0,0,0,1,0,0,0,0
